The code bellow takes a set of audio (speech) files and a set of files containing background noise and merges each sample speech file with each sample of background noise.

Source of the formulas: https://medium.com/analytics-vidhya/adding-noise-to-audio-clips-5d8cee24ccb8

In [1]:
# Imported libraries
import librosa
import numpy as np
import math
from scipy.io.wavfile import write
import regex as re

The add_background_noise() function uses the librosa library to load both files, adjust the length of the noise file, calculate the RMS to adjust the noise sample and merge both files.

The required parameters are:

- **s_path:** a string containing the path for the speech file (.wav format)

- **n_path:** a string containing the path for the noise file (.wav format)
- **offset:** a float representing the starting point, in seconds, to start loading the noise file (default = 0)
- **SNR:** an integer representing the Signal to Noise Ratio (default = 10)

The function returns the merged audios.

In [2]:
def add_background_noise(s_path, n_path, offset=0.0, SNR=10):
  sample, sr = librosa.load(s_path)  # Load audio file
  duration_s = librosa.get_duration(y=sample, sr=sr)  # Get audio file length
  noise, sr = librosa.load(n_path, offset=offset, duration=duration_s) # Load same length of noise file
  RMS_s=math.sqrt(np.mean(sample**2))  # Calculate RMS for audio sample
  RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))  # Calculate required RMS for noise
  RMS_n_current=math.sqrt(np.mean(noise**2))  # Calculate current RMS for noise
  noise=noise*(RMS_n/RMS_n_current)  # Adjust noise
  sample_noise = sample + noise  # Add sample to noise
  sample_noise = librosa.resample(sample_noise, orig_sr=sr,target_sr=16000)
  return sample_noise


**s_list** is a list of strings, each containing the path for one of the sample speech files.

**n_list** is a list of list. Each inner list has the path for a background noise .wav file at the index 0 and the offset for the audio sample at the index 1.

The loop goes though both lists, extracts and combines the names of both files, calls the add_background_noise() function, and saves the merged .wav file in the determined path, using the combined name previously saved.

In [14]:
s_list = [
    "LJ048-0226.wav", 
    "LJ007-0015.wav",
    ]
n_list = [
    ["hallway_ch01.wav", 222.0]
    ]

for n_path in n_list:
  for s_path in s_list:
    file_name = re.findall('(\S+?)\.wav', s_path)[0] + "_" + re.findall('(\S+?)\.wav', n_path[0])[0] + "snr30.wav"
    sr = librosa.get_samplerate(n_path[0])
    write("/content/" + file_name, rate=16000, data=add_background_noise(s_path, n_path[0], offset=n_path[1], SNR=30))